<a href="https://colab.research.google.com/github/AndresPautrat/Concurrente/blob/main/ExamenParcial_Promela.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Explicación del procedimiento de verificación:
Para verificar se puede usar el metodo assert(condicion) el cual verifica si la expresion dentro de la funcion siempre se cumpla, en caso que falle aparecera el mensaje assertion violated.

La segunda opcion es usar comandos para que promela busque errores por su propia cuenta.

#Comandos para realizar la verificación.
para usar assert se tiene que correr el codigo: !spin -run nombre.pml
este comando tambien da la verificacion automatica

#Explicación de cómo realizar la interpretación.
Esto depende de si se usa el assert o la deteccion automatica, si se usa el assert como programador se estara buscando un error especifico y se sabra directamente que se debe interpretar.
Si se usa la deteccion automatica debera verse la linea de codigo y analisarse. tambien podrian hacerse pruebas con prints para verificar si es un deadlook y nunca imprime, si el resultado siempre es el mismo o si las secciones criticas se causan conflictos entre si.

In [ ]:
!apt install spin

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  spin
0 upgraded, 1 newly installed, 0 to remove and 22 not upgraded.
Need to get 559 kB of archives.
After this operation, 1,776 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 spin amd64 6.4.6+dfsg-2 [559 kB]
Fetched 559 kB in 1s (600 kB/s)
Selecting previously unselected package spin.
(Reading database ... 153160 files and directories currently installed.)
Preparing to unpack .../spin_6.4.6+dfsg-2_amd64.deb ...
Unpacking spin (6.4.6+dfsg-2) ...
Setting up spin (6.4.6+dfsg-2) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [ ]:
%%writefile exclusion.pml
bool wantp=false
bool wantq=false
int count
active proctype P(){
    do
    :: wantq==false->
        wantp=true
        count++
        assert(count<2)
        count--
        wantp=false
    od
}
active proctype Q(){
    do
    :: wantp==false->
        wantq=true
        count++
        assert(count<2)
        count--
        wantq=false
    od
}

Writing exclusion.pml


Aqui vemos como ambas secciones criticas entran al mismo tiempo y el contador llega a ser 2 cuando no deberia pasar de 1

In [ ]:
!spin -run exclusion.pml

pan:1: assertion violated (count<2) (at depth 7)
pan: wrote exclusion.pml.trail

(Spin Version 6.4.6 -- 2 December 2016)
	+ Partial Order Reduction

Full statespace search for:
	never claim         	- (none specified)
	assertion violations	+
	cycle checks       	- (disabled by -DSAFETY)
	invalid end states	+

State-vector 28 byte, depth reached 11, errors: 1
       22 states, stored
       10 states, matched
       32 transitions (= stored+matched)
        0 atomic steps
hash conflicts:         0 (resolved)

Stats on memory usage (in Megabytes):
    0.001	equivalent memory usage for states (stored*(State-vector + overhead))
    0.291	actual memory usage for states
  128.000	memory used for hash table (-w24)
    0.534	memory used for DFS stack (-m10000)
  128.730	total actual memory usage



pan: elapsed time 0 seconds


In [ ]:
%%writefile deadlock.pml
#define wait(s) atomic { s > 0 -> s-- }
#define signal(s) s++

byte forks[5] = { 1, 1, 1, 1, 1 }

active[5] proctype Philosopher() {
    byte i = _pid
    do
    ::
        printf("%d thinking\n", i)

        wait(forks[i])
        assert(forks[0]+forks[1]+forks[2]+forks[3]+forks[4]<5)
        wait(forks[(i+1) % 5])

        printf("%d eat\n", i)

        signal(forks[i])
        signal(forks[(i+1) % 5])
    od
}


Overwriting deadlock.pml


En este caso puede suceder que todos los procesos entren a la linea wait(forks[i]) al mismo tiempo y por lo tanto no podran pasar de la linea wait(forks[(i+1) % 5]) y todos los procesos se trabaran.

In [ ]:
!spin -run deadlock.pml

pan:1: invalid end state (at depth 68)
pan: wrote deadlock.pml.trail

(Spin Version 6.4.6 -- 2 December 2016)
	+ Partial Order Reduction

Full statespace search for:
	never claim         	- (none specified)
	assertion violations	+
	cycle checks       	- (disabled by -DSAFETY)
	invalid end states	+

State-vector 52 byte, depth reached 69, errors: 1
       70 states, stored
       13 states, matched
       83 transitions (= stored+matched)
        0 atomic steps
hash conflicts:         0 (resolved)

Stats on memory usage (in Megabytes):
    0.005	equivalent memory usage for states (stored*(State-vector + overhead))
    0.291	actual memory usage for states
  128.000	memory used for hash table (-w24)
    0.534	memory used for DFS stack (-m10000)
  128.730	total actual memory usage



pan: elapsed time 0 seconds


In [ ]:
%%writefile livelock.pml
#define wait(s) atomic { s > 0 -> s-- }
#define signal(s) s++

byte roomEmpty = 1
byte readers = 0
byte mutex = 1

active proctype Writer() {
    do
    ::
        wait(roomEmpty)
        printf("Writer writing!\n")
        signal(roomEmpty)
    od
}

active[3] proctype Reader() {
    do
    ::
        wait(mutex)
        readers++
        if
        :: readers == 1 -> wait(roomEmpty)
        :: else -> skip
        fi
        signal(mutex)

        printf("Student%d 1\n", _pid)

        wait(mutex)
        readers--
        if
        :: readers == 0 -> signal(roomEmpty)
        :: else -> skip
        fi
        signal(mutex)
    od
}

Writing livelock.pml


Este es una ejemplo de live lock ya que puede suceder que los Readers siempre entren y no permitan que los Writer tomen su turno.

In [ ]:
!spin -run livelock.pml


(Spin Version 6.4.6 -- 2 December 2016)
	+ Partial Order Reduction

Full statespace search for:
	never claim         	- (none specified)
	assertion violations	+
	cycle checks       	- (disabled by -DSAFETY)
	invalid end states	+

State-vector 44 byte, depth reached 79, errors: 0
      145 states, stored
       70 states, matched
      215 transitions (= stored+matched)
        0 atomic steps
hash conflicts:         0 (resolved)

Stats on memory usage (in Megabytes):
    0.010	equivalent memory usage for states (stored*(State-vector + overhead))
    0.289	actual memory usage for states
  128.000	memory used for hash table (-w24)
    0.534	memory used for DFS stack (-m10000)
  128.730	total actual memory usage


unreached in proctype Writer
	livelock.pml:15, state 9, "-end-"
	(1 of 9 states)
unreached in proctype Reader
	livelock.pml:38, state 29, "-end-"
	(1 of 29 states)

pan: elapsed time 0 seconds
